<a href="https://colab.research.google.com/github/apriliap/Implementasi-Deep-Learning-Menggunakan-CNN-Untuk-Klasifikasi-Jenis-Awan/blob/main/Model_CNN_Arsitektur_VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sn;
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle

import cv2
import tensorflow as tf
from tqdm import tqdm

from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

In [ ]:
#persiapan data dan konfigurasi sebelum proses pelatihan model
class_names = ['cumulonimbus', 'cumulus', 'stratocumulus','nimbostratus', 'stratus']
class_names_label = {class_name:i for i, class_name in enumerate(class_names)}

nb_classes = len(class_names)

IMAGE_SIZE = (224, 224)

In [ ]:
def load_data():
    datasets = [r'/content/drive/MyDrive/dataset_yoga/dataset_awan_1442',
                r'/content/drive/MyDrive/dataset_yoga/dataset_awan_1442']
    output = []

    for dataset in datasets:

        images = []
        labels = []

        print("Loading {}".format(dataset))

        # ulangi setiap folder yang sesuai dengan kategori
        for folder in os.listdir(dataset):
            label = class_names_label[folder]

            # ulangi setiap gambar di folder
            for file in tqdm(os.listdir(os.path.join(dataset, folder))):

                # dapatkan nama jalur gambar
                img_path = os.path.join(os.path.join(dataset, folder), file)

                # pemrosesan gambar
                image = cv2.imread(img_path)
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                image = cv2.resize(image, IMAGE_SIZE)

                # tambahkan gambar dan label yang sesuai ke output
                images.append(image)
                labels.append(label)

        images = np.array(images, dtype = 'float32')
        labels = np.array(labels, dtype = 'int32')

        output.append((images, labels))

    return output

In [ ]:
#memuat data gambar dan label dari dataset
(train_images, train_labels), (test_images, test_labels) = load_data()

In [ ]:
#fungsi shuffle untuk mengacak urutan data dalam train_images dan train_labels
train_images, train_labels = shuffle(train_images, train_labels, random_state=5)

In [ ]:
#menampilkan informasi ukuran dataset dan ukuran gambar
n_train = train_images.shape[0]
n_test = test_images.shape[0]

print("Training Examples = {}".format(n_train))
print("Testing Examples = {}".format(n_test))
print("Image Size = {}".format(IMAGE_SIZE))

In [ ]:
import pandas as pd
_,train_counts = np.unique(train_labels, return_counts=True)
_,test_counts = np.unique(test_labels, return_counts=True)

pd.DataFrame({'train':train_counts, 'test':test_counts}, index=class_names).plot.bar()
plt.show()

In [ ]:
plt.pie(train_counts, labels=class_names, autopct='%1.1f%%')
plt.show()

In [ ]:
#mengubah rentang nilai piksel gambar yang awalnya berada dalam rentang 0 hingga 255 menjadi rentang 0 hingga 1
train_images /= 255.0
test_images /= 255.0

In [ ]:
def display_random_image(class_names, images, labels):
    index = np.random.randint(images.shape[0])

    plt.figure()
    plt.imshow(images[index])
    plt.xticks([])
    plt.yticks([])
    plt.title("Image #{}".format(index+1)+" "+class_names[labels[index]])
    plt.show()

In [ ]:
display_random_image(class_names, train_images, train_labels)

In [ ]:
def display_examples(class_names, images, labels):
    fig = plt.figure(figsize=(10,10))
    fig.suptitle("Examples",fontsize=12)
    for i in range(25):
        plt.subplot(5,5,i+1)
        plt.imshow(images[i])
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.xlabel(class_names[labels[i]])
    plt.show()

In [ ]:
display_examples(class_names, train_images, train_labels)

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing import image

model = VGG16(weights='imagenet', include_top=False)

model.summary()

In [ ]:
#ekstraksi fitur
train_features = model.predict(train_images)
test_features = model.predict(test_images)

In [ ]:
# Memuat Model VGG16 Tanpa Layer Klasifikasi Teratas
vgg16_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Membekukan Layer Awal dan Membuka Layer Terakhir
for layer in vgg16_base.layers[:-5]:  # Example: unfreeze the last 5 layers
    layer.trainable = False

#  Membangun Model Klasifikasi Baru
model = Sequential([
    vgg16_base,
    Flatten(),
    Dense(50, activation='relu'),
    Dense(5, activation='softmax')
])

# Pelatihan Model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(train_images, train_labels, batch_size=32, epochs=30, validation_split=0.2)


In [ ]:
import matplotlib.pyplot as plt
accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(accuracy))

plt.plot(epochs, accuracy, "b", label="trainning accuracy")
plt.plot(epochs, val_accuracy, "r", label="validation accuracy")
plt.legend()
plt.show()

plt.plot(epochs, loss, "b", label="trainning loss")
plt.plot(epochs, val_loss, "r", label="validation loss")
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# Prediksi label untuk data uji
test_predictions = model.predict(test_images)  # Use original test images
test_predictions_labels = np.argmax(test_predictions, axis=1)

# Buat confusion matrix
cm = confusion_matrix(test_labels, test_predictions_labels)

# Plot confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, cmap='Blues', fmt='g', cbar=False,
            xticklabels=class_names, yticklabels=class_names)
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
from sklearn.metrics import classification_report
import numpy as np

# Prediksi pada data uji
test_prediction = model.predict(test_images)
test_prediction_label = np.argmax(test_prediction, axis=1)

# Daftar nama kelas (sesuaikan dengan label Anda)
class_names = ['cumulonimbus', 'cumulus', 'stratocumulus','nimbostratus', 'stratus']

# Laporan klasifikasi dengan nama kelas
print(classification_report(test_labels, test_prediction_label, target_names=class_names))

In [ ]:
print(model.input_shape)

In [ ]:
class_names = ['cumulonimbus', 'cumulus', 'stratocumulus','nimbostratus', 'stratus']
def awan_prediction(new_image):
    # Muat dan preprocess gambar
    img_array = load_and_preprocess_image(new_image, target_size=(224,224))

    # Tampilkan gambar
    test_image = image.load_img(new_image, target_size=(224, 224))
    plt.axis("off")
    plt.imshow(test_image)
    plt.show()

    # Ekstraksi fitur dari VGG16
    features = vgg16_model.predict(img_array)

    # Prediksi label menggunakan model yang telah dilatih
    predicted_array = model.predict(features)
    pred_labels = class_names[np.argmax(predicted_array)]
    predicted_accuracy = round(np.max(predicted_array) * 100, 2)

    print("awan is", pred_labels, "with", predicted_accuracy, "% accuracy")


In [ ]:
# menyimpan model
model.save('model-vgg16.h5')